In [1]:
import json
data_path = "../../data/train.json"
with open(data_path, "r") as f:
    dataset = json.load(f)

In [2]:
query_list = {}
for ind, data in dataset.items():
    query = data['query']    
    if query in query_list:
        print(query)
        break
    else:
        query_list[query] = data

In [3]:
list(query_list.keys())[:10]

['#HASH#value excel',
 '%2f',
 '%localappdata%',
 '*.pst file location',
 '.22 hornet',
 '.22 pellets',
 '.223 vs 5.56',
 '.338 lapua magnum',
 '.338 winchester magnum',
 '.45 acp']

In [4]:
"%2f" in query_list

True

In [5]:
", ".join(dataset['0']['facet'])

'2010, 2013, 2016'

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# model_path = "/home1/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/upstage/llama-30b-instruct-2048" # "upstage/llama-30b-instruct-2048"
model_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/HyperbeeAI/Tulpar-7b-v0" # "upstage/llama-30b-instruct-2048"

tokenizer = AutoTokenizer.from_pretrained(model_path) 
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:31<00:00, 15.96s/it]


In [8]:
prompt = "### User:\nThe facet for 'caesars atlantic city' is 'caesars atlantic city events'. Tell me the rationale for it in one sentence.\n\n### Assistant:\n"
print(prompt)

### User:
The facet for 'caesars atlantic city' is 'caesars atlantic city events'. Tell me the rationale for it in one sentence.

### Assistant:



In [17]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

output = model.generate(**inputs, streamer=streamer, use_cache=True, max_new_tokens=100, temperature=0.01)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

The facet 'caesars atlantic city events' is included to provide information about the various events and entertainment options available at Caesars Atlantic City, attracting guests and promoting the casino's overall experience.


In [4]:
import json
with open("LLM_train.jsonl", "r", encoding="utf-8") as f:
    dataset = f.readlines()

In [9]:
json.loads(dataset[4000])

{'query': '2015 volvo xc60',
 'facet_info': [['specs',
   'The 2015 Volvo XC60 features a 2.0L turbocharged engine, all-wheel drive, and a sleek Scandinavian design, offering a comfortable and luxurious driving experience.'],
  ['manual',
   'The 2015 Volvo XC60 is available with a manual transmission option.'],
  ['review',
   'The 2015 Volvo XC60 is a luxury crossover SUV that offers a comfortable ride, advanced safety features, and a stylish interior, making it a top choice for those seeking a reliable and well-designed vehicle in its class.'],
  ['for sale', 'The 2015 Volvo XC60 is currently for sale.'],
  ['accessories',
   "The 2015 Volvo XC60 offers a variety of accessories to enhance the vehicle's appearance and functionality, such as roof racks, wheel upgrades, and interior convenience features."]]}